In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# American Option Pricing with the Binomial Model

This notebook contains my solution to **Module 5** of the Coursera course  
[*Introduction to Financial Engineering and Risk Management*](https://www.coursera.org/learn/financial-engineering-intro/home/info)

The module is dedicated to the **Binomial Model for option pricing**, and this notebook specifically focuses on **American options**, emphasizing the treatment of **early exercise** using a recombining binomial tree. The goal is to provide a clear and educational implementation that closely follows the theoretical framework presented in the course.

In [2]:
N = 15
T = 0.25
S0 = 100 
r = 0.02
sigm = 0.3
c = 0.01
K  = 110.0

In [3]:
def crr_ud(sigm, T, N):
    dt = T / N
    u = np.exp(sigm * np.sqrt(dt))   # no rounding
    d = 1.0 / u
    return u, d, dt

In [4]:
u, d, dt = crr_ud(sigm, T, N)

In [5]:
u,d, dt

(np.float64(1.0394896104013376),
 np.float64(0.9620105771080376),
 0.016666666666666666)

In [6]:
def stock_tree(S0, u, d, N, c):
    """
    Build full binomial stock price tree matrix.

    Rows = time step i = 0..N
    Cols = number of down moves j = 0..N
    Valid entries satisfy j <= i. Others are NaN.

    Returns: (N+1, N+1) numpy array
    """
    tree = np.full((N + 1, N + 1), np.nan, dtype=float)
    # tree[0,0] = S0

    for i in range(N + 1):
        j = np.arange(i + 1)              # 0..i        
        tree[i, j] = S0 * (u ** (i - j)) * (d ** j) # + c * S0

    return tree

In [7]:
S = stock_tree(S0, u, d, N, c)

In [8]:
S

array([[100.        ,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [103.94896104,  96.20105771,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [108.05386501, 100.        ,  92.54643505,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan],
       [112.32087004, 103.94896104,  96.20105771,  89.03064939,
                 nan,          nan,          nan,          nan,
                 nan,          nan,          nan,          nan,
                 nan,          nan,  

In [9]:
for i in range(N + 1):
    vals = [f"{S[i, j]:8.4f}" for j in range(i + 1)]
    print(f"t={i:2d}: ", " ".join(vals))

t= 0:  100.0000
t= 1:  103.9490  96.2011
t= 2:  108.0539 100.0000  92.5464
t= 3:  112.3209 103.9490  96.2011  89.0306
t= 4:  116.7564 108.0539 100.0000  92.5464  85.6484
t= 5:  121.3670 112.3209 103.9490  96.2011  89.0306  82.3947
t= 6:  126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646
t= 7:  131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  82.3947  76.2534
t= 8:  136.3205 126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646  73.3565
t= 9:  141.7038 131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  82.3947  76.2534  70.5698
t=10:  147.2996 136.3205 126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646  73.3565  67.8889
t=11:  153.1164 141.7038 131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  82.3947  76.2534  70.5698  65.3098
t=12:  159.1629 147.2996 136.3205 126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646  73.3565  67.8889  62.8287
t=13:  165.4482 153.1164 141.7038 131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  8

In [10]:
def get_p(r, c, T, N, u, d):
    dt = T / N
    return ( np.exp((r -c)* dt) - d ) / (u - d)

In [11]:
p = get_p(r, c, T, N, u, d)

In [12]:
p

np.float64(0.4924700506245105)

In [13]:
def american_call_option(r, c, T, N, u, d, S, K):
    p = get_p(r, c, T, N, u, d)

    disc = np.exp(-r * dt)
    N = S.shape[0] - 1

    # terminal payoff (valid nodes j=0..N)
    V = np.maximum(S[N, :N+1] - K, 0.0)

    print("\nV = ", V)

    # backward induction
    for i in range(N - 1, -1, -1):
        # continuation value for nodes j=0..i
        cont = disc * (p * V[:i+1] + (1 - p) * V[1:i+2])

        print("\ncont = ", cont)

        # immediate exercise value at time i
        exer = np.maximum(S[i, :i+1] - K, 0.0)

        # American value
        V = np.maximum(cont, exer)

    return V

In [14]:
cv = american_call_option(r, c, T, N, u, d, S, K)


V =  [68.77315076 55.44817785 43.11639045 31.70376083 21.14177898 11.3670413
  2.32087004  0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]

cont =  [61.98966127 49.17303267 37.31169981 26.3344591  16.17541416  6.77358022
  1.14257807  0.          0.          0.          0.          0.
  0.          0.          0.        ]

cont =  [55.46634654 43.13866905 31.72984301 21.17138123 11.39990126  3.91437298
  0.56249795  0.          0.          0.          0.          0.
  0.          0.        ]

cont =  [49.19328051 37.33590077 26.36231853 16.20665937  7.5982383   2.2124584
  0.27692107  0.          0.          0.          0.          0.
  0.        ]

cont =  [43.1609158  31.75589207 21.20094919 11.83368264  4.86317237  1.22970527
  0.13632988  0.          0.          0.          0.          0.        ]

cont =  [37.36006926 26.39014427 16.44129953  8.29317507  3.01807281  0.67455962
  0.06711601  0.          0.          0.          0

In [15]:
round(cv[0],2)

np.float64(2.6)

In [16]:
def american_put_option(r, c, T, N, u, d, S, K):
    p = get_p(r, c, T, N, u, d)

    disc = np.exp(-r * dt)
    N = S.shape[0] - 1

    # terminal payoff (valid nodes j=0..N)
    V = np.maximum(K - S[N, :N+1] , 0.0)

    # backward induction
    for i in range(N - 1, -1, -1):
        # continuation value for nodes j=0..i
        cont = disc * (p * V[:i+1] + (1 - p) * V[1:i+2])

        # immediate exercise value at time i
        exer = np.maximum( K - S[i, :i+1], 0.0)

        print("\n i= : ", i)
        print("Immediate exercise value: ", exer)
        print("Continue value: ", cont)

        # American value
        V = np.maximum(cont, exer)

    return V

In [17]:
pv = american_put_option(r, c, T, N, u, d, S, K)


 i= :  14
Immediate exercise value:  [ 0.          0.          0.          0.          0.          0.
  1.94613499 10.         17.45356495 24.3515736  30.73543469 36.64347055
 42.11114712 47.17128687 51.85426581]
Continue value:  [ 0.          0.          0.          0.          0.          0.
  3.07005997  9.98000472 17.43232752 24.32918659 30.7119838  36.61903507
 42.08580043 47.14509689 51.8272954 ]

 i= :  13
Immediate exercise value:  [ 0.          0.          0.          0.          0.          0.
  6.05103896 13.79894229 20.96935061 27.60530789 33.74664979 39.43024277
 44.69020547 49.55811342]
Continue value:  [ 0.          0.          0.          0.          0.          1.55762809
  6.58501671 13.77831391 20.94752726 27.58237864 33.72269706 39.40534286
 44.66442897 49.53152567]

 i= :  12
Immediate exercise value:  [ 0.          0.          0.          0.          0.          1.94613499
 10.         17.45356495 24.3515736  30.73543469 36.64347055 42.11114712
 47.17128687]
Cont

In [18]:
round(pv[0],2)

np.float64(12.36)

In [19]:
pv[0]

np.float64(12.359784797284908)

#### let's see if they satisfy put call parity

In [20]:
pv + S0 *np.exp(-c * T) == cv + K *np.exp(-r * T)

array([False])

# Compute the fair value of an American call option with strike K=110K=110K, equals, 110 and maturity n=10n=10n, equals, 10 periods where the option is written on a futures contract that expires after 15 periods. The futures contract is on the same underlying security of the previous questions.

In [21]:
def american_call_on_futures_from_stock_tree(S, K, u, d, r, c, T, N, opt_periods=10, fut_periods=15):

    dt = T/N
    p = get_p(r, c, T, N, u, d)
    disc = np.exp(-r * dt)

    # build futures tree only up to option maturity (0..opt_periods)
    F = np.full((opt_periods + 1, opt_periods + 1), np.nan)
    for i in range(opt_periods + 1):
        carry = np.exp((r - c) * (fut_periods - i) * dt)  # time from i to futures expiry
        F[i, :i+1] = S[i, :i+1] * carry


    # terminal payoff at option expiry
    V = np.maximum(F[opt_periods, :opt_periods+1] - K, 0.0)

    # backward induction with early exercise
    for i in range(opt_periods - 1, -1, -1):
        cont = disc * (p * V[:i+1] + (1 - p) * V[1:i+2])
        exer = np.maximum(F[i, :i+1] - K, 0.0)
        V = np.maximum(cont, exer)

        print("\n i= : ", i)
        print("Immediate exercise value: ", exer)
        print("Continue value: ", cont)

    return V

In [22]:
FV = american_call_on_futures_from_stock_tree(S, K, u, d, r, c, T, N, opt_periods=10, fut_periods=15)


 i= :  9
Immediate exercise value:  [31.84553547 21.27298635 11.48846905  2.43324709  0.          0.
  0.          0.          0.          0.        ]
Continue value:  [31.83492206 21.26589653 11.48464019  3.37412447  0.          0.
  0.          0.          0.          0.        ]

 i= :  8
Immediate exercise value:  [26.47965015 16.30705078  6.89267271  0.          0.          0.
  0.          0.          0.        ]
Continue value:  [26.47082507 16.301616    7.36773983  1.66110145  0.          0.
  0.          0.          0.        ]

 i= :  7
Immediate exercise value:  [21.3167513  11.52897195  2.47073109  0.          0.          0.
  0.          0.        ]
Continue value:  [21.3096469  11.76616004  4.46995971  0.81777008  0.          0.
  0.          0.        ]

 i= :  6
Immediate exercise value:  [16.34916015  6.93164343  0.          0.          0.          0.
  0.        ]
Continue value:  [16.46405127  8.06043259  2.61549212  0.40259305  0.          0.
  0.        ]

 i= :  

In [23]:
round(FV[0],2)

np.float64(1.66)